# Network Dissection (for generators)

In this notebook, we will examine internal layer representations for a generator trained via GAN.

Setup matplotlib, torch, and numpy for a high-resolution browser.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
import matplotlib as mpl
from importlib import reload
import IPython
mpl.rcParams['lines.linewidth'] = 0.25
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.linewidth'] = 0.25

Set up experiment directory and settings

In [ ]:
import torch, argparse, os, shutil, inspect, json, numpy
import netdissect
from netdissect.easydict import EasyDict
from netdissect import pbar, nethook, renormalize, parallelfolder, pidfile
from netdissect import upsample, tally, imgviz, imgsave, bargraph, show
from experiment import dissect_experiment as experiment
# choices are alexnet, vgg16, or resnet152.
args = EasyDict(model='progan', dataset='kitchen', seg='netpqc', layer='layer5', quantile=0.01)
resdir = 'results/%s-%s-%s-%s-%s' % (args.model, args.dataset, args.seg, args.layer, int(args.quantile * 1000))
def resfile(f):
    return os.path.join(resdir, f)

load generator model and z dataset

In [ ]:
from netdissect import zdataset
model = experiment.load_model(args)
layername = experiment.instrumented_layername(args)
model.retain_layer(layername)
# dataset = experiment.load_dataset(args, model.model)
dataset = zdataset.z_dataset_for_model(model, size=10000, seed=1)

upfn = experiment.make_upfn(args, dataset, model, layername)
sample_size = len(dataset)
percent_level = 1.0 - args.quantile

print('Inspecting layer %s of model %s on %s' % (layername, args.model, args.dataset))

Load segmenter, segment labels, classifier labels

In [ ]:
# Classifier labels
segmodel, seglabels, segcatlabels = experiment.setting.load_segmenter(args.seg)


Test generator to make some images

In [ ]:
from netdissect import renormalize

indices = [200, 755, 489, 709, 423, 60, 100, 110, 120]
batch = torch.cat([dataset[i][0][None,...] for i in indices])
outs = model(batch.cuda())
imgs = [renormalize.as_image(t) for t in outs]
show([[img] for img in imgs])


segment single image, and visualize the labels

In [ ]:
from netdissect import imgviz

iv = imgviz.ImageVisualizer(120)
renorm = renormalize.renormalizer(target='zc')
seg = segmodel.segment_batch(renorm(outs).cuda(), downsample=4)

show([(iv.image(outs[i]), iv.segmentation(seg[i,0]),
            iv.segment_key(seg[i,0], segmodel))
            for i in range(len(seg))])

visualize activations for single layer of single image

In [ ]:
from netdissect import imgviz

acts = model.retained_layer(layername).cpu()
ivsmall = imgviz.ImageVisualizer((64,  64))
display(show.blocks(
    [[[ivsmall.masked_image(outs[0], acts, (0, u))],
      [ivsmall.heatmap(acts, (0, u), mode='nearest')]] for u in range(min(acts.shape[1], 12))]
))

num_units = acts.shape[1]

## Collect quantile statistics

First, unconditional quantiles over the activations.  We will upsample them to 56x56 to match with segmentations later.


In [ ]:
pbar.descnext('rq')
def compute_samples(batch, *args):
    z_batch = batch.cuda()
    _ = model(z_batch)
    acts = model.retained_layer(layername)
    hacts = upfn(acts)
    return hacts.permute(0, 2, 3, 1).contiguous().view(-1, acts.shape[1])
rq = tally.tally_quantile(compute_samples, dataset,
                          sample_size=sample_size,
                          r=8192,
                          num_workers=100,
                          pin_memory=True,
                          cachefile=resfile('rq.npz'))

## Visualize Units

Collect topk stats first.

In [ ]:
pbar.descnext('topk')
def compute_image_max(batch, *args):
    image_batch = batch.cuda()
    _ = model(image_batch)
    acts = model.retained_layer(layername)
    acts = acts.view(acts.shape[0], acts.shape[1], -1)
    acts = acts.max(2)[0]
    return acts
topk = tally.tally_topk(compute_image_max, dataset, sample_size=sample_size,
        batch_size=50, num_workers=30, pin_memory=True,
        cachefile=resfile('topk.npz'))

Then we just need to run through and visualize the images.

In [ ]:
pbar.descnext('unit_images')

iv = imgviz.ImageVisualizer((256, 256), image_size=256, source='zc', quantiles=rq,
        level=rq.quantiles(percent_level))
def compute_acts(data_batch):
    data_batch = data_batch.cuda()
    image_batch = model(data_batch)
    acts_batch = model.retained_layer(layername)
    return acts_batch, image_batch
unit_images = iv.masked_images_for_topk(
        compute_acts, dataset, topk, k=4, num_workers=30, pin_memory=True,
        cachefile=resfile('big-top4images.npz'))

In [ ]:
for u in [15, 35]:
    print('unit %d' % u)
    display(unit_images[u])

In [ ]:
selected_unit = 314
selected_segment = seglabels.index('window')

for i in [0, 10, 20, 30, 31, 33]:
    batch = dataset[i][0][None,...].cuda()
    img = model(batch)[0]
    seg = segmodel.segment_batch(img[None,...])
    seg = seg[0]
    seg[seg != selected_segment] = 0
    acts = model.retained_layer(layername)[0]
    show([i, [iv.image(img)], [iv.segmentation(seg)], [iv.heatmap(acts, selected_unit, mode='nearest')]])


## Label Units

Collect 99 quantile stats.

In [ ]:
# Use the segmodel for segmentations.  With broden, we could use ground truth instead.
def compute_conditional_indicator(batch, *args):
    image_batch = batch.cuda()
    seg = segmodel.segment_batch(renorm(image_batch), downsample=4)
    _ = model(image_batch)
    acts = model.retained_layer(layername)
    hacts = upfn(acts)
    iacts = (hacts > level_at_99).float() # indicator
    return tally.conditional_samples(iacts, seg)
pbar.descnext('condi99')
condi99 = tally.tally_conditional_mean(compute_conditional_indicator,
        dataset, sample_size=sample_size,
        num_workers=3, pin_memory=True,
        cachefile=resfile('condi99.npz'))

In [ ]:
iou_99 = tally.iou_from_conditional_indicator_mean(condi99)
unit_label_99 = [
        (concept.item(), seglabels[concept], segcatlabels[concept], bestiou.item())
        for (bestiou, concept) in zip(*iou_99.max(0))]
label_list = [labelcat for concept, label, labelcat, iou in unit_label_99 if iou > 0.04]
display(IPython.display.SVG(experiment.graph_conceptcatlist(label_list)))
len(label_list)

Show a few units with their labels

In [ ]:
top_iou_units = sorted(enumerate(unit_label_99), key=lambda x: -x[1][-1])
for urec in top_iou_units[:10]:
    u = urec[0]
    print('unit %d, label %s, iou %.3f' % (u, unit_label_99[u][1], unit_label_99[u][3]))
    display(unit_images[u])